# MultiLayer Perceptron sem Framework

#### Definição do problema
- **Objetivo**: identificar se uma pessoa é atleta ou não;
- **Features**: altura e peso de uma pessoa;
- **Saída**: 1 (é atleta) ou 0 (não é atleta).

#### Estrutura da MLP
- **Camada de entrada**: 2 neurônios (altura e peso);
- **Camada oculta**: 3 neurônios (com função de ativação ReLU);
- **Camada de saída**: 1 neurônio (com função de ativação Sigmoid).

##### Passos
0. **Inicializar os dados de entrada**: definir os dados para treinamento e os rótulos com as respostas;
1. **Inicializar os pesos e bias**: pesos e bias com valores aleatórios pequenos;
1. **Forward Propagation**: propagar a entrada até a saída;
1. **Calcular o erro**: usar erro quadrático médio (MSE) ou entropia cruzada;
1. **Backward Propagation**: calcular os gradientes e ajustar pesos e bias;
1. **Loop de treinamento**: repetir várias vezes para minimizar o erro.

<center><image src="../Imagens/exemploMLP_3_perceptrons_na_camada_oculta.jpg" width="400"></image></center>

### Passo 0: Dados e rótulos

In [6]:
import numpy as np

# Dados de entrada (altura, peso)
X = np.array([
    [1.70, 65],
    [1.80, 80],
    [1.60, 50],
    [1.90, 90]
])

# Rótulos corretos (y) - só para quando for calcular o erro
y = np.array([
    [1],
    [1],
    [0],
    [1]
])

### Passo 1: Inicializar os pesos e bias

In [7]:
# Fixar a semente para reprodutibilidade, ou seja, garante o retorno do mesmo número aleatório
np.random.seed(42)

# Inicializar pesos e bias

# W1: pesos da camada de entrada para a camada oculta
# 2 entradas (altura, peso) → 3 neurônios ocultos
W1 = np.random.randn(2, 3) * 0.01

# b1: bias para cada um dos 3 neurônios ocultos
b1 = np.zeros((1, 3))

# W2: pesos da camada oculta para a camada de saída
# 3 neurônios ocultos → 1 saída
W2 = np.random.randn(3, 1) * 0.01

# b2: bias para a saída
b2 = np.zeros((1, 1))

# Mostrar os valores iniciais
print("Pesos W1 (entrada -> camada oculta):")
print(W1)
print("\nBias b1 (camada oculta):")
print(b1)
print("\nPesos W2 (camada oculta -> saída):")
print(W2)
print("\nBias b2 (saída):")
print(b2)

Pesos W1 (entrada -> camada oculta):
[[ 0.00496714 -0.00138264  0.00647689]
 [ 0.0152303  -0.00234153 -0.00234137]]

Bias b1 (camada oculta):
[[0. 0. 0.]]

Pesos W2 (camada oculta -> saída):
[[ 0.01579213]
 [ 0.00767435]
 [-0.00469474]]

Bias b2 (saída):
[[0.]]


### Passo 2: Forward Propagation

In [10]:
# Funções de ativação
def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# (X, W1, b1, W2, b2 já devem ter sido definidos no passo anterior)

# Forward propagation
# 1. Cálculo na camada oculta
z1 = np.dot(X, W1) + b1        # Produto dos dados de entrada pelos pesos + bias da camada oculta
a1 = relu(z1)                  # Aplicação da função de ativação ReLU

# 2. Cálculo na camada de saída
z2 = np.dot(a1, W2) + b2       # Produto da saída da camada oculta pelos pesos + bias da camada de saída
a2 = sigmoid(z2)               # Aplicação da função de ativação Sigmoid para gerar a saída final

# Resultado
print("Saída da camada oculta (a1):")
print(a1)
print("\nSaída final da rede (a2):")
print(a2)

Saída da camada oculta (a1):
[[0.99841355 0.         0.        ]
 [1.22736474 0.         0.        ]
 [0.76946235 0.         0.        ]
 [1.38016444 0.         0.        ]]

Saída final da rede (a2):
[[0.50394169]
 [0.50484552]
 [0.50303782]
 [0.50544872]]


### Passo 3: Calcular o erro

Nesta etapa, podem ser utilizados os métodos **Erro Quadrático Médio (MSE)** ou a **Entropia Cruzada (Cross-Entropy)**. Para fins didáticos, usaremos o MSE. 

- O MSE mede o quão longe as previsões da rede estão dos valores corretos;
- Ele calcula a diferença entre a resposta correta (rótulo) e a resposta da rede (previsão);
- Essa diferença é elevada ao quadrado (para não ter número negativo) e depois tira a média de todas as diferenças.


In [12]:
# Função para calcular o erro quadrático médio (MSE)
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

# Cálculo do erro
loss = mean_squared_error(y, a2)

print(f"\nErro (MSE) atual: {loss:.6f}")


Erro (MSE) atual: 0.247220


### Passo 4: Backward Propagation

In [14]:
# Derivadas das funções de ativação
def relu_derivative(x):
    return (x > 0).astype(float)

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

# Backward propagation

# 1. Derivada do erro em relação à saída (a2)
# Para MSE: dL/da2 = (a2 - y)
d_loss_a2 = (a2 - y)

# 2. Derivada em relação a z2 (entrada da camada de saída)
dz2 = d_loss_a2 * sigmoid_derivative(z2)

# 3. Derivadas dos pesos e bias da camada de saída
dW2 = np.dot(a1.T, dz2)  # Gradiente dos pesos W2
db2 = np.sum(dz2, axis=0, keepdims=True)  # Gradiente do bias b2

# 4. Derivada em relação a a1 (saída da camada oculta)
d_a1 = np.dot(dz2, W2.T)

# 5. Derivada em relação a z1 (entrada da camada oculta)
dz1 = d_a1 * relu_derivative(z1)

# 6. Derivadas dos pesos e bias da camada oculta
dW1 = np.dot(X.T, dz1)  # Gradiente dos pesos W1
db1 = np.sum(dz1, axis=0, keepdims=True)  # Gradiente do bias b1

# Mostrar os gradientes (opcional, para entender)
print("Gradiente dW2:")
print(dW2)
print("\nGradiente db2:")
print(db2)
print("\nGradiente dW1:")
print(dW1)
print("\nGradiente db1:")
print(db1)

Gradiente dW2:
[[-0.34958632]
 [ 0.        ]
 [ 0.        ]]

Gradiente db2:
[[-0.24565219]]

Gradiente dW1:
[[-0.00737945  0.          0.        ]
 [-0.36007549  0.          0.        ]]

Gradiente db1:
[[-0.00387937  0.          0.        ]]


### Passo 5: Loop de treinamento

In [15]:
# Hiperparâmetros
learning_rate = 0.01
epochs = 1000  # Número de vezes que a rede vai treinar

# Loop de treinamento
for epoch in range(epochs):
    # --- Forward propagation ---
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    
    # --- Cálculo do erro ---
    loss = mean_squared_error(y, a2)
    
    # --- Backward propagation ---
    d_loss_a2 = (a2 - y)
    dz2 = d_loss_a2 * sigmoid_derivative(z2)
    dW2 = np.dot(a1.T, dz2)
    db2 = np.sum(dz2, axis=0, keepdims=True)
    
    d_a1 = np.dot(dz2, W2.T)
    dz1 = d_a1 * relu_derivative(z1)
    dW1 = np.dot(X.T, dz1)
    db1 = np.sum(dz1, axis=0, keepdims=True)
    
    # --- Atualização dos pesos e bias ---
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    
    # --- Mostrar o erro a cada 100 épocas ---
    if epoch % 100 == 0:
        print(f"Época {epoch} - Erro (MSE): {loss:.6f}")

# Mostrar a saída final da rede após o treinamento
print("\nSaída final da rede após treinamento:")
print(a2)

Época 0 - Erro (MSE): 0.247220
Época 100 - Erro (MSE): 0.155880
Época 200 - Erro (MSE): 0.154595
Época 300 - Erro (MSE): 0.153306
Época 400 - Erro (MSE): 0.152011
Época 500 - Erro (MSE): 0.150712
Época 600 - Erro (MSE): 0.149407
Época 700 - Erro (MSE): 0.148096
Época 800 - Erro (MSE): 0.146779
Época 900 - Erro (MSE): 0.145455

Saída final da rede após treinamento:
[[0.74553067]
 [0.80803484]
 [0.6709623 ]
 [0.84268967]]


In [18]:
# Threshold (limiar) para classificar
threshold = 0.5

# Classificação: 1 se >= 0.5, senão 0
classificacao = (a2 >= threshold).astype(int)

# Exibir resultados
print("\nClassificação final (0 = não atleta, 1 = atleta):")
print(classificacao)

# Comparação
print("Comparação previsão x realidade:\n")
for i in range(len(y)):
    print(f"Exemplo {i+1}: Previsto = {classificacao[i][0]} | Real = {y[i][0]} | {'✅ Acertou' if classificacao[i][0] == y[i][0] else '❌ Errou'}")


Classificação final (0 = não atleta, 1 = atleta):
[[1]
 [1]
 [1]
 [1]]
Comparação previsão x realidade:

Exemplo 1: Previsto = 1 | Real = 1 | ✅ Acertou
Exemplo 2: Previsto = 1 | Real = 1 | ✅ Acertou
Exemplo 3: Previsto = 1 | Real = 0 | ❌ Errou
Exemplo 4: Previsto = 1 | Real = 1 | ✅ Acertou


#### O que isso significa?
- O modelo não aprendeu perfeitamente.
- Há um erro residual (uma pequena quantidade de erro ainda ficou no modelo).
- Isso é normal em redes simples ou em casos com poucos dados!
- A rede não conseguiu separar 100% os exemplos corretos.

#### Possíveis causas
- Poucos dados: só 4 exemplos de treinamento. Redes neurais precisam de muitos dados para generalizar bem;
- Rede muito simples: só uma camada oculta pequena (3 neurônios);
- Épocas insuficientes: talvez precisasse de mais tempo de treinamento;
- Dados muito próximos: altura e peso da Pessoa 3 podem ser parecidos com de atletas; 
- Aprendizado raso: usamos MSE como função de erro - para classificação binária, entropia cruzada seria melhor.


# Atualização do Modelo Acima

### Passo 0: dados e rótulos

In [20]:
import numpy as np

# Fixar a semente para reprodutibilidade
np.random.seed(42)

# Gerar dados para atletas (altura entre 1.75 e 2.00 metros, peso entre 65 e 85 kg)
altura_atleta = np.random.uniform(1.75, 2.00, 50)
peso_atleta = np.random.uniform(65, 85, 50)

# Gerar dados para não atletas (altura entre 1.50 e 1.75 metros, peso entre 70 e 100 kg)
altura_nao_atleta = np.random.uniform(1.50, 1.75, 50)
peso_nao_atleta = np.random.uniform(70, 100, 50)

# Juntar os dados
altura = np.concatenate((altura_atleta, altura_nao_atleta))
peso = np.concatenate((peso_atleta, peso_nao_atleta))

# Montar X (entradas)
X = np.column_stack((altura, peso))

# Montar y (rótulos)
# 1 para atletas, 0 para não atletas
y = np.array([[1]] * 50 + [[0]] * 50)

# Embaralhar os dados (importante!)
indices = np.arange(X.shape[0])
np.random.shuffle(indices)

X = X[indices]
y = y[indices]

# Mostrar amostra
print("Exemplos de X (altura, peso):")
print(X[:5])

print("\nExemplos de y (rótulos):")
print(y[:5])

Exemplos de X (altura, peso):
[[ 1.65239108 91.77867037]
 [ 1.99247746 70.42698064]
 [ 1.72689162 99.56951362]
 [ 1.6591026  77.18685672]
 [ 1.87379423 79.26489574]]

Exemplos de y (rótulos):
[[0]
 [1]
 [0]
 [0]
 [1]]


### Passo 1: Inicializar os pesos e bias

In [21]:
# Fixar a semente para reprodutibilidade
np.random.seed(42)

# Inicializar pesos e bias

# W1: pesos da camada de entrada para a camada oculta
# 2 entradas (altura, peso) → 4 neurônios ocultos
W1 = np.random.randn(2, 4) * 0.01  # Pequenos valores aleatórios

# b1: bias para cada um dos 4 neurônios ocultos
b1 = np.zeros((1, 4))

# W2: pesos da camada oculta para a camada de saída
# 4 neurônios ocultos → 1 saída
W2 = np.random.randn(4, 1) * 0.01

# b2: bias para a saída
b2 = np.zeros((1, 1))

# Mostrar os valores iniciais (opcional para conferência)
print("Pesos W1 (entrada -> camada oculta):")
print(W1)

print("\nBias b1 (camada oculta):")
print(b1)

print("\nPesos W2 (camada oculta -> saída):")
print(W2)

print("\nBias b2 (saída):")
print(b2)

Pesos W1 (entrada -> camada oculta):
[[ 0.00496714 -0.00138264  0.00647689  0.0152303 ]
 [-0.00234153 -0.00234137  0.01579213  0.00767435]]

Bias b1 (camada oculta):
[[0. 0. 0. 0.]]

Pesos W2 (camada oculta -> saída):
[[-0.00469474]
 [ 0.0054256 ]
 [-0.00463418]
 [-0.0046573 ]]

Bias b2 (saída):
[[0.]]


### Passo 2: Forward Propagation

In [22]:
# Funções de ativação
def relu(x):
    return np.maximum(0, x)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# (X, W1, b1, W2, b2 já devem ter sido definidos no passo anterior)

# Forward propagation
# 1. Cálculo na camada oculta
z1 = np.dot(X, W1) + b1        # Produto dos dados de entrada pelos pesos + bias da camada oculta
a1 = relu(z1)                  # Aplicação da função de ativação ReLU

# 2. Cálculo na camada de saída
z2 = np.dot(a1, W2) + b2       # Produto da saída da camada oculta pelos pesos + bias da camada de saída
a2 = sigmoid(z2)               # Aplicação da função de ativação Sigmoid para gerar a saída final

# Mostrar resultados
print("Saída da camada oculta (a1):")
print(a1)

print("\nSaída final da rede (a2):")
print(a2)

Saída da camada oculta (a1):
[[0.         0.         1.46008287 0.7295078 ]
 [0.         0.         1.12509695 0.57082713]
 [0.         0.         1.5835994  0.7904321 ]
 [0.         0.         1.22969055 0.61762737]
 [0.         0.         1.26389774 0.63684478]
 [0.         0.         1.20952637 0.60948069]
 [0.         0.         1.31915035 0.66426644]
 [0.         0.         1.41681316 0.70944746]
 [0.         0.         1.30092137 0.65585557]
 [0.         0.         1.18431926 0.5946059 ]
 [0.         0.         1.19846439 0.60182046]
 [0.         0.         1.2313187  0.61943165]
 [0.         0.         1.16061704 0.58522086]
 [0.         0.         1.05308092 0.53471619]
 [0.         0.         1.41498899 0.70623966]
 [0.         0.         1.18768116 0.59964034]
 [0.         0.         1.0721989  0.54274937]
 [0.         0.         1.29977924 0.65236529]
 [0.         0.         1.43426232 0.71635837]
 [0.         0.         1.28418223 0.64807828]
 [0.         0.         1.12684

### Passo 3: Calcular o erro

In [23]:
# Função para calcular entropia cruzada
def cross_entropy_loss(y_true, y_pred):
    epsilon = 1e-15  # Pequeno valor para evitar log(0)
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)  # Garante que y_pred esteja entre (0,1)
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

# Cálculo do erro
loss = cross_entropy_loss(y, a2)

# Mostrar o erro
print(f"\nErro (Entropia Cruzada) atual: {loss:.6f}")


Erro (Entropia Cruzada) atual: 0.692874


### Passo 4: Backward Propagation

In [24]:
# Backward propagation

# 1. Derivada do erro em relação à saída a2
# (já é simplificado para Cross-Entropy com Sigmoid)
d_loss_a2 = a2 - y

# 2. Derivadas dos pesos e bias da camada de saída
dW2 = np.dot(a1.T, d_loss_a2) / X.shape[0]  # Normaliza pela quantidade de exemplos
db2 = np.sum(d_loss_a2, axis=0, keepdims=True) / X.shape[0]

# 3. Derivada em relação a saída da camada oculta (a1)
d_a1 = np.dot(d_loss_a2, W2.T)

# 4. Derivada em relação a z1 (entrada da camada oculta)
def relu_derivative(x):
    return (x > 0).astype(float)

dz1 = d_a1 * relu_derivative(z1)

# 5. Derivadas dos pesos e bias da camada oculta
dW1 = np.dot(X.T, dz1) / X.shape[0]  # Normaliza pela quantidade de exemplos
db1 = np.sum(dz1, axis=0, keepdims=True) / X.shape[0]

# Mostrar os gradientes (opcional para estudo)
print("Gradiente dW2:")
print(dW2)

print("\nGradiente db2:")
print(db2)

print("\nGradiente dW1:")
print(dW1)

print("\nGradiente db1:")
print(db1)

Gradiente dW2:
[[0.        ]
 [0.        ]
 [0.03870161]
 [0.01803009]]

Gradiente db2:
[[-0.00222799]]

Gradiente dW1:
[[ 0.          0.          0.00029814  0.00029963]
 [ 0.          0.         -0.01147921 -0.01153648]]

Gradiente db1:
[[0.00000000e+00 0.00000000e+00 1.03249112e-05 1.03764237e-05]]


### Passo 5: Loop de treinamento

In [25]:
# Hiperparâmetros
learning_rate = 0.1
epochs = 1000

# Loop de treinamento
for epoch in range(epochs):
    # --- Forward propagation ---
    z1 = np.dot(X, W1) + b1
    a1 = relu(z1)
    z2 = np.dot(a1, W2) + b2
    a2 = sigmoid(z2)
    
    # --- Cálculo do erro (Cross-Entropy) ---
    loss = cross_entropy_loss(y, a2)
    
    # --- Backward propagation ---
    d_loss_a2 = a2 - y
    dW2 = np.dot(a1.T, d_loss_a2) / X.shape[0]
    db2 = np.sum(d_loss_a2, axis=0, keepdims=True) / X.shape[0]
    d_a1 = np.dot(d_loss_a2, W2.T)
    dz1 = d_a1 * relu_derivative(z1)
    dW1 = np.dot(X.T, dz1) / X.shape[0]
    db1 = np.sum(dz1, axis=0, keepdims=True) / X.shape[0]
    
    # --- Atualização dos pesos e bias ---
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    
    # --- Exibir o erro a cada 100 épocas ---
    if epoch % 100 == 0:
        print(f"Época {epoch} - Erro (Cross-Entropy): {loss:.6f}")

# Mostrar a saída final da rede após o treinamento
print("\nSaída final da rede após o treinamento (a2):")
print(a2)

Época 0 - Erro (Cross-Entropy): 0.692874
Época 100 - Erro (Cross-Entropy): 0.680755
Época 200 - Erro (Cross-Entropy): 0.673913
Época 300 - Erro (Cross-Entropy): 0.673947
Época 400 - Erro (Cross-Entropy): 0.671096
Época 500 - Erro (Cross-Entropy): 0.685659
Época 600 - Erro (Cross-Entropy): 0.718397
Época 700 - Erro (Cross-Entropy): 0.724689
Época 800 - Erro (Cross-Entropy): 0.694257
Época 900 - Erro (Cross-Entropy): 0.698852

Saída final da rede após o treinamento (a2):
[[0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.50436836]
 [0.5043

### Teste com o novo modelo

In [27]:
def descobrir_atleta(pessoas):
    # Normalizar os novos dados
    # Usar a mesma média e desvio padrão dos dados de treino
    mean_X = X.mean(axis=0)
    std_X = X.std(axis=0)
    pessoas_normalizadas = (pessoas - mean_X) / std_X

    # Forward propagation nos novos dados
    z1_novos = np.dot(pessoas_normalizadas, W1) + b1
    a1_novos = relu(z1_novos)
    z2_novos = np.dot(a1_novos, W2) + b2
    a2_novos = sigmoid(z2_novos)

    # Classificar como atleta (1) ou não (0)
    threshold = 0.5
    classificacao_novos = (a2_novos >= threshold).astype(int)

    # Mostrar resultados
    print("Probabilidades previstas para novas pessoas:")
    print(a2_novos)

    print("\nClassificação final para novas pessoas (0 = não atleta, 1 = atleta):")
    print(classificacao_novos)

In [30]:
# Novos dados (altura, peso)
pessoas = np.array([
    [1.78, 75],  # Pessoa 1
    [1.60, 80],  # Pessoa 2
    [1.85, 68],  # Pessoa 3
    [1.70, 90]   # Pessoa 4
])

descobrir_atleta(pessoas)

Probabilidades previstas para novas pessoas:
[[0.50425725]
 [0.49864281]
 [0.50425375]
 [0.50425916]]

Classificação final para novas pessoas (0 = não atleta, 1 = atleta):
[[1]
 [0]
 [1]
 [1]]


# Desafio

Construa MLPs para os seguintes cenários:
- Identificar se um e-mail é spam ou não (classificação binária). Features:
    - número de palavras,
	- presença de palavras-chave como "promoção", "grátis",
	- quantidade de links,
	- tamanho do e-mail.
- Classificar pacientes como "em risco" ou "não em risco" baseado em exames médicos. Features: 
	- Com base em dados médicos fictícios:
	- pressão arterial,
	- colesterol,
	- glicemia,
	- idade,
	- IMC.